## <font color="green"> Human Protein Atlas </font>
<font color="blue"> Read the data from the HPA project </font>

In [1]:
import numpy as np
import pandas as pd
from Bio import SeqIO
import csv

In [2]:
#import urllib2
import urllib.request
fhand = urllib.request.urlopen('https://www.proteinatlas.org/download/normal_tissue.tsv.zip')
#response = urllib2.urlopen('https://www.proteinatlas.org/download/normal_tissue.tsv.zip')
zipcontent= fhand.read()
print(type(fhand))
print(type(zipcontent))

<class 'http.client.HTTPResponse'>
<class 'bytes'>


In [3]:
import requests, zipfile, io
r = requests.get('https://www.proteinatlas.org/download/normal_tissue.tsv.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("/Users/saheeba/Downloads/")
#filepath= "/Users/saheeba/Downloads/Human_atlas_subcellular_location.tsv"
HP_atlas = pd.read_csv(z.open('normal_tissue.tsv'), sep = "\t")
print(HP_atlas.shape)
HP_atlas.head()

(1056061, 6)


,Gene,Gene name,Tissue,Cell type,Level,Reliability
0,ENSG00000000003,TSPAN6,adipose tissue,adipocytes,Not detected,Approved
1,ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected,Approved
2,ENSG00000000003,TSPAN6,appendix,glandular cells,Medium,Approved
3,ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected,Approved
4,ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected,Approved


In [4]:
bingo_new = pd.read_csv('genesofinterest_human_atlas.csv')
genes = bingo_new.iloc[:,0].tolist()
print(len(genes))
print("Done!")

975
Done!


In [5]:
protein_atlas = HP_atlas.loc[HP_atlas['Gene name'].isin(genes)]
print(protein_atlas.shape)
export_csv = protein_atlas.to_csv (r'/Users/saheeba/Downloads/Corresponding_human_atlas.csv', index = None, header=True)
protein_atlas.head()


(48993, 6)


,Gene,Gene name,Tissue,Cell type,Level,Reliability
865,ENSG00000001561,ENPP4,adipose tissue,adipocytes,Low,Approved
866,ENSG00000001561,ENPP4,adrenal gland,glandular cells,High,Approved
867,ENSG00000001561,ENPP4,appendix,glandular cells,Medium,Approved
868,ENSG00000001561,ENPP4,appendix,lymphoid tissue,Low,Approved
869,ENSG00000001561,ENPP4,bone marrow,hematopoietic cells,Medium,Approved


In [6]:
protein_atlas.loc[protein_atlas['Gene name'].isin(genes)]
sorted_atlas = protein_atlas.sort_values(by=['Gene name'], ascending=True)
print(sorted_atlas.shape)
sorted_atlas.head(3)

(48993, 6)


,Gene,Gene name,Tissue,Cell type,Level,Reliability
629557,ENSG00000159640,ACE,soft tissue 1,fibroblasts,Not detected,Enhanced
629514,ENSG00000159640,ACE,colon,glandular cells,Not detected,Enhanced
629515,ENSG00000159640,ACE,colon,peripheral nerve/ganglion,Not detected,Enhanced


#### <font color="blue">Conditional combine all culoumns and stack them together | dataframes

In [7]:
custom_seqs = pd.read_excel('customisedMappingID.xlsx')
sorted_custom_seq = custom_seqs.sort_values('Gene names',ascending=True)
sorted_custom_seq.rename(columns={'yourlist:M202001296746803381A1F0E0DB47453E0216320D6E4B519':'positionID'},inplace=True)
sorted_custom_seq.head()

,positionID,Entry,Entry name,Status,Protein names,Gene names,Organism,Length
172,P12821[30-1256],P12821,ACE_HUMAN,reviewed,Angiotensin-converting enzyme (ACE) (EC 3.2.1....,ACE DCP DCP1,Homo sapiens (Human),1306
794,Q9BYF1[18-740],Q9BYF1,ACE2_HUMAN,reviewed,Angiotensin-converting enzyme 2 (EC 3.4.17.23)...,ACE2 UNQ868/PRO1885,Homo sapiens (Human),805
273,Q04771[21-123],Q04771,ACVR1_HUMAN,reviewed,Activin receptor type-1 (EC 2.7.11.30) (Activi...,ACVR1 ACVRLK2,Homo sapiens (Human),509
931,D3DPA4[1-145],D3DPA4,D3DPA4_HUMAN,unreviewed,Serine/threonine-protein kinase receptor (EC 2...,ACVR1 hCG_1811747,Homo sapiens (Human),509
87,P36896[24-126],P36896,ACV1B_HUMAN,reviewed,Activin receptor type-1B (EC 2.7.11.30) (Activ...,ACVR1B ACVRLK4 ALK4,Homo sapiens (Human),505


In [8]:
df = sorted_atlas.groupby(['Gene name','Level'], as_index=False)['Cell type'].agg(', '.join).set_index(['Gene name','Level'])['Cell type'].unstack().reset_index()
print(df.shape) 
df.head(3)


(619, 5)


Level,Gene name,High,Low,Medium,Not detected
0,ACE,"glandular cells, glandular cells, glandular ce...",endothelial cells,"glandular cells, pneumocytes, cells in seminif...","fibroblasts, glandular cells, peripheral nerve..."
1,ACE2,"glandular cells, Leydig cells, cells in semini...","glandular cells, glandular cells, glandular ce...",NaN,"adipocytes, smooth muscle cells, epidermal cel..."
2,ACVR1,"glandular cells, smooth muscle cells","squamous epithelial cells, glial cells, pneumo...","myocytes, glandular cells, glandular cells, gl...","cells in endometrial stroma, glandular cells, ..."


In [9]:
sorted_custom_seq['Gene name'] = genes
s1 = pd.merge(sorted_custom_seq,df ,how='inner', on=['Gene name'])
export = s1.to_csv('merged_expression.csv',index = None, header=True)
print(s1.shape)
s1.head(3)

(645, 13)


,positionID,Entry,Entry name,Status,Protein names,Gene names,Organism,Length,Gene name,High,Low,Medium,Not detected
0,P12821[30-1256],P12821,ACE_HUMAN,reviewed,Angiotensin-converting enzyme (ACE) (EC 3.2.1....,ACE DCP DCP1,Homo sapiens (Human),1306,ACE,"glandular cells, glandular cells, glandular ce...",endothelial cells,"glandular cells, pneumocytes, cells in seminif...","fibroblasts, glandular cells, peripheral nerve..."
1,Q9BYF1[18-740],Q9BYF1,ACE2_HUMAN,reviewed,Angiotensin-converting enzyme 2 (EC 3.4.17.23)...,ACE2 UNQ868/PRO1885,Homo sapiens (Human),805,ACE2,"glandular cells, Leydig cells, cells in semini...","glandular cells, glandular cells, glandular ce...",NaN,"adipocytes, smooth muscle cells, epidermal cel..."
2,Q04771[21-123],Q04771,ACVR1_HUMAN,reviewed,Activin receptor type-1 (EC 2.7.11.30) (Activi...,ACVR1 ACVRLK2,Homo sapiens (Human),509,ACVR1,"glandular cells, smooth muscle cells","squamous epithelial cells, glial cells, pneumo...","myocytes, glandular cells, glandular cells, gl...","cells in endometrial stroma, glandular cells, ..."


In [10]:
s2 = s1.drop(['Status','Organism','Gene name','Entry name'], axis = 1)
s2.rename(columns={'High':'Protein Expression : High','Low':'Protein Expression : Low','Medium':'Protein Expression : Med'},inplace=True)
s2.head()

,positionID,Entry,Protein names,Gene names,Length,Protein Expression : High,Protein Expression : Low,Protein Expression : Med,Not detected
0,P12821[30-1256],P12821,Angiotensin-converting enzyme (ACE) (EC 3.2.1....,ACE DCP DCP1,1306,"glandular cells, glandular cells, glandular ce...",endothelial cells,"glandular cells, pneumocytes, cells in seminif...","fibroblasts, glandular cells, peripheral nerve..."
1,Q9BYF1[18-740],Q9BYF1,Angiotensin-converting enzyme 2 (EC 3.4.17.23)...,ACE2 UNQ868/PRO1885,805,"glandular cells, Leydig cells, cells in semini...","glandular cells, glandular cells, glandular ce...",NaN,"adipocytes, smooth muscle cells, epidermal cel..."
2,Q04771[21-123],Q04771,Activin receptor type-1 (EC 2.7.11.30) (Activi...,ACVR1 ACVRLK2,509,"glandular cells, smooth muscle cells","squamous epithelial cells, glial cells, pneumo...","myocytes, glandular cells, glandular cells, gl...","cells in endometrial stroma, glandular cells, ..."
3,D3DPA4[1-145],D3DPA4,Serine/threonine-protein kinase receptor (EC 2...,ACVR1 hCG_1811747,509,"glandular cells, smooth muscle cells","squamous epithelial cells, glial cells, pneumo...","myocytes, glandular cells, glandular cells, gl...","cells in endometrial stroma, glandular cells, ..."
4,P36896[24-126],P36896,Activin receptor type-1B (EC 2.7.11.30) (Activ...,ACVR1B ACVRLK4 ALK4,505,NaN,"glandular cells, decidual cells, squamous epit...","exocrine glandular cells, glandular cells, gla...","melanocytes, Langerhans, keratinocytes, fibrob..."


### <font color = 'blue'> To add corresponding sequences to the table </font>

In [11]:
identifiers = []
sequences = []
lengths = []
for seq_record in SeqIO.parse('/Users/saheeba/Downloads/splits_files/out.fasta', 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequences.append(str(seq_record.seq))#sequences.append(repr(seq_record.seq))#sequences.append(seq_record.seq)#.upper()
        lengths.append(len(seq_record.seq))
        s11 = pd.Series(identifiers, name='ID')
        s12 = pd.Series(sequences, name='Seq')
        s13 = pd.Series(lengths, name='len')
        pdf = pd.DataFrame(dict(ID=s11, sequences=s12, Seq_length=s13))
pdf['Entry']= pdf['ID'].str.split('|').str[1]
pdf.head()

,ID,sequences,Seq_length,Entry
0,sp|Q9NR97|27-827,EENFSRSYPCDEKKQNDSVIAECSNRRLQEVPQTVGKYVTELDLSD...,801,Q9NR97
1,sp|Q02846|52-462,AVFTVGVLGPWACDPIFSRARPDLAARLAAARLNRDPGLAGGPRFE...,411,Q02846
2,sp|Q9NY15|26-2478,QVLFKGCDVKTTFVTHVPCTSCAAIKKQTCPSGWLRELPDQITQDC...,2453,Q9NY15
3,sp|A6NFA1|20-494,RPQPPDGGQCRPPGSQRDLNSFLWTIRRDPPAYLFGTIHVPYTRVW...,475,A6NFA1
4,sp|Q86V40|20-477,ASRRGAPGTANCELKPQQSELNSFLWTIKRDPPSYFFGTIHVPYTR...,458,Q86V40


In [12]:
s3 = pd.merge(s2,pdf ,how='inner', on=['Entry'])
#s3.drop(['Status','Organism','Gene name','Entry name','Length','ID'], axis = 1)
s3.head()
#s3.rename(columns={'High':'Protein Expression : High','Low':'Protein Expression : Low','Medium':'Protein Expression : Medium'},inplace=True)
s3 = s3[['positionID','Entry','sequences','Protein names','Gene names','Seq_length','Protein Expression : High','Protein Expression : Low','Protein Expression : Med','Not detected']]
print(s3.shape)
s3.head(3)

(645, 10)


,positionID,Entry,sequences,Protein names,Gene names,Seq_length,Protein Expression : High,Protein Expression : Low,Protein Expression : Med,Not detected
0,P12821[30-1256],P12821,LDPGLQPGNFSADEAGAQLFAQSYNSSAEQVLFQSVAASWAHDTNI...,Angiotensin-converting enzyme (ACE) (EC 3.2.1....,ACE DCP DCP1,1227,"glandular cells, glandular cells, glandular ce...",endothelial cells,"glandular cells, pneumocytes, cells in seminif...","fibroblasts, glandular cells, peripheral nerve..."
1,Q9BYF1[18-740],Q9BYF1,QSTIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMN...,Angiotensin-converting enzyme 2 (EC 3.4.17.23)...,ACE2 UNQ868/PRO1885,723,"glandular cells, Leydig cells, cells in semini...","glandular cells, glandular cells, glandular ce...",NaN,"adipocytes, smooth muscle cells, epidermal cel..."
2,Q04771[21-123],Q04771,MEDEKPKVNPKLYMCVCEGLSCGNEDHCEGQQCFSSLSINDGFHVY...,Activin receptor type-1 (EC 2.7.11.30) (Activi...,ACVR1 ACVRLK2,103,"glandular cells, smooth muscle cells","squamous epithelial cells, glial cells, pneumo...","myocytes, glandular cells, glandular cells, gl...","cells in endometrial stroma, glandular cells, ..."


### <font color = 'blue'> Signaling pathway data: To add corresponding info to the table </font>

In [29]:
SignaLink = pd.read_csv('signal_Book.csv')
SignaLink.rename(columns={'source_uniprotAC':'Source','target_uniprotAC':'Target'},inplace=True)
print(SignaLink.shape)
SignaLink.head(2)

(87581, 20)


,source_name,Source,source_speciesID,source_species,source_topology,source_pathways,target_name,Target,target_speciesID,target_species,target_topology,target_pathways,layer,interaction_type,directness,effect,references,source,confidence_score,score_from_the_source
0,NFKB1,P19838,ENSG00000109320,H. sapiens,Transcription factor,NaN,PRKCE,Q02156,ENSG00000171132,H. sapiens,Scaffold,NaN,Transcriptional regulation,Transcriptional directed,indirect,unknown,14681366,"JASPAR(url: http://jaspar.cgb.ki.se/ ,pmid: 14...",GO Semantic Similarity: 0.563594,"Jaspar: 10.1804, Jaspar: 10.1804, Jaspar: 10.3..."
1,NFKB1,P19838,ENSG00000109320,H. sapiens,Transcription factor,NaN,PAG1,Q9NWQ8,ENSG00000076641,H. sapiens,"Co-factor,Scaffold",RTK(non-core),Transcriptional regulation,Transcriptional directed,indirect,unknown,14681366,"JASPAR(url: http://jaspar.cgb.ki.se/ ,pmid: 14...",GO Semantic Similarity: 0.521359,"Jaspar: 10.1804, Jaspar: 10.1804"


In [28]:
'''SignaLink_1_0 = SignaLink.groupby(['Entry','target_pathways'], as_index=True)['Target'].agg(', '.join).unstack().reset_index()#.unstack().reset_index().set_index(['Entry','target_pathways'])['Target']
SignaLink_1_0.head(3)'''

"SignaLink_1_0 = SignaLink.groupby(['Entry','target_pathways'], as_index=True)['Target'].agg(', '.join).unstack().reset_index()#.unstack().reset_index().set_index(['Entry','target_pathways'])['Target']\nSignaLink_1_0.head(3)"

In [59]:
SignaLink_source = SignaLink[['Source', 'source_pathways']]
print(SignaLink_source.shape)
source_match= SignaLink_source.groupby(['Source','source_pathways'], as_index=False)['source_pathways'].agg(",".join)
print(source_match.shape)
source_match.head(12)
SignaLink_target = SignaLink[['Target', 'target_pathways']]
target_match= SignaLink_target.groupby(['Target','target_pathways'], as_index=True)['target_pathways'].agg(",".join)
#print(SignaLink_target.shape)
print(target_match.shape)
target_match.head(3)

(87581, 2)
(430,)
(752,)


Target  target_pathways
A7KAX9  RTK(non-core)      RTK(non-core),RTK(non-core),RTK(non-core),RTK(...
A7MBM2  Hedgehog(core)     Hedgehog(core),Hedgehog(core),Hedgehog(core),H...
C9JF09  RTK(non-core)                                          RTK(non-core)
Name: target_pathways, dtype: object

In [27]:
signal_data = SignaLink_1_0.loc[SignaLink_1_0['Entry'].isin(s2['Entry'])]
signal_pathway=pd.merge(s2,SignaLink_1_0 ,how='inner', on=['Entry'])
export_csv = signal_pathway.to_csv (r'/Users/saheeba/Downloads/SignaLink_sorted.csv', index = None, header=True)
print(signal_pathway.shape)
signal_pathway.head(2)

(71, 53)


,positionID,Entry,Protein names,Gene names,Length,Protein Expression : High,Protein Expression : Low,Protein Expression : Med,Not detected,Hedgehog(core),...,"RTK(non-core),TGF(core)","RTK(non-core),TGF(non-core)","RTK(non-core),TGF(non-core),TGF(core),WNT/Wingless(core)","RTK(non-core),WNT/Wingless(non-core)",TGF(core),TGF(non-core),"TGF(non-core),WNT/Wingless(core)","TGF(non-core),WNT/Wingless(non-core)",WNT/Wingless(core),WNT/Wingless(non-core)
0,Q04771[21-123],Q04771,Activin receptor type-1 (EC 2.7.11.30) (Activi...,ACVR1 ACVRLK2,509,"glandular cells, smooth muscle cells","squamous epithelial cells, glial cells, pneumo...","myocytes, glandular cells, glandular cells, gl...","cells in endometrial stroma, glandular cells, ...",NaN,...,NaN,NaN,NaN,NaN,"Q15797, Q99717",NaN,NaN,NaN,NaN,NaN
1,P36896[24-126],P36896,Activin receptor type-1B (EC 2.7.11.30) (Activ...,ACVR1B ACVRLK4 ALK4,505,NaN,"glandular cells, decidual cells, squamous epit...","exocrine glandular cells, glandular cells, gla...","melanocytes, Langerhans, keratinocytes, fibrob...",NaN,...,NaN,NaN,NaN,NaN,Q04771,NaN,NaN,NaN,NaN,NaN


### <font color='blue'> xtalkdb: a database of signaling pathway crosstalk </font>
xtalkdb contains curated information for 650 pairs of pathways from over 1600 publications. In addition, the database reports the molecular components (e.g. proteins, hormones, microRNAs) that mediate crosstalk between a pair of pathways and the species and tissue in which the crosstalk was observed

In [13]:
X_talk_db=pd.read_csv('X_talkdb.csv')
print(X_talk_db.shape)
X_talk_db.rename(columns={'Molecule A Identifier':'Prot1','Molecule B Identifier':'Prot2'},inplace=True)
X_talk_db.head(2) 

(3474, 20)


,Pathway A,Pathway B,Pubmed Query,PMID,Crosstalk,Transcriptional,Regulation type,Molecule A,Prot1,Molecule A Source,Molecule B,Prot2,Molecule B Source,Species,Tissue Name,BTO ID,Condition,Sentence from paper,Additional notes,Misleading sentences
0,Adipocytokine signaling pathway,ErbB signaling pathway,Adipocytokine ErbB,25838314,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mentions both pathways but does not say taht t...,NaN
1,Adipocytokine signaling pathway,ErbB signaling pathway,Adipocytokine ErbB,23817051,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,It is not mentioned that the pathways interact...,NaN


In [ ]:
#To check if we miss anything in the X_talk database due to multiple entries in one row 
'''import re
#re = re.split('; |, |\/|:',str)
list_prot1 = X_talk_db['Prot1'].str.split('; |, |\/|:').tolist()
s3.loc[s3['Entry'].isin(list_prot1)]
true_false = pd.DataFrame(list_prot1).isin(s3['Entry'])
true_false.to_csv('true_false_lits.txt')
'''

In [14]:
#Extract signaling pathways for each protein ID in X_talkdb

X_talkprot1 = X_talk_db.groupby(['Prot1','Crosstalk'], as_index=False)['Pathway A'].agg(', '.join).set_index(['Prot1','Crosstalk'])['Pathway A'].unstack().reset_index()
X_talkprot2 = X_talk_db.groupby(['Prot2','Crosstalk'], as_index=False)['Pathway B'].agg(', '.join).set_index(['Prot2','Crosstalk'])['Pathway B'].unstack().reset_index()
X_talkprot1.rename(columns={'Prot1':'Entry'},inplace=True)
X_talkprot2.rename(columns={'Prot2':'Entry'},inplace=True)
print(X_talkprot1.shape)
print(X_talkprot2.shape)
X_talkprot2.head()

(268, 4)
(312, 4)


Crosstalk,Entry,no,unclear,yes
0,A4K2Y1,NaN,NaN,Hippo signaling pathway
1,B0LXP3,NaN,NaN,NF-kappa B signaling pathway
2,B6V8E6,NaN,NaN,Wnt signaling pathway
3,C4PCF3/A5HNF6,NaN,Toll-like receptor signaling pathway,NaN
4,D3ZDU2,NaN,NaN,mTOR signaling pathway


In [ ]:
X_talkprot3 = pd.merge(X_talkprot2,X_talkprot1, how='outer',on=None, left_on=None, right_on=None,
         left_index=True, right_index=True, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=True,
         validate="1:1")
print(X_talkprot3.shape)
X_talkprot3.head(20)

In [ ]:
X_talkprot3 = X_talk_db.groupby(['Prot1','Prot2'], as_index=False)['Pathway A'].agg(', '.join).set_index(['Prot1','Prot2'])['Pathway A'].unstack().reset_index(drop=True)

'''
X_talkprot3 = pd.merge(X_talkprot1,X_talkprot2, how='inner',on=None, left_on=None, right_on=None,
         left_index=True, right_index=True, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=True,
         validate="1:1")
print(X_talkprot3.shape)
X_talkprot3.head()
'''
X_talkprot3.head(2)

In [ ]:
print(X_talkprot1.shape)
print(X_talkprot2.shape)
X_talkprot1.head()
crosstalk = pd.concat([X_talkprot1,X_talkprot2])
print(crosstalk.shape)
crosstalk.head()

In [ ]:
crosstalk = crosstalk.loc[crosstalk['Entry'].isin(s2['Entry'])]
print(crosstalk.shape)
crosstalk.head().reset_index(drop=True)

In [ ]:
#another way of doing same match again
prot1_pathway = X_talkprot1.loc[X_talkprot1['Entry'].isin(s2['Entry'])]
prot2_pathway = X_talkprot2.loc[X_talkprot2['Entry'].isin(s2['Entry'])]
prot1_prot2 = pd.concat([prot1_pathway,prot2_pathway])
print(prot1_prot2.shape)
#prot1_prot2['Entry'] = prot1_prot2['Entry'].str.split(" ").str[0]#.str.split(" ") 
prot1_prot2.head()#['Crosstalk']#.drop(['Crosstalk'], axis = 1)

## <font color='green'> Extracting signaling pathway from Uniprot dataset </font>


In [ ]:
!egrep -w 'signaling|pathway' 997_from\ uniprot.gff > signaling_pathway_uniprot.csv
!egrep -w 'signaling' 997_from\ uniprot.gff > signaling_uniprot.csv
!egrep -w 'pathway' 997_from\ uniprot.gff > pathway_uniprot.csv

In [ ]:
signaling_pathway_uniprot = pd.read_csv('signaling_pathway_uniprot.csv',sep="\t",header= None)
print(signaling_pathway_uniprot.shape)
signaling_uniprot = pd.read_csv('signaling_uniprot.csv',sep="\t",header= None)
print(signaling_uniprot.shape)
pathway_uniprot = pd.read_csv('pathway_uniprot.csv',sep="\t",header= None)
print(pathway_uniprot.shape)
signaling_pathway_uniprot

In [ ]:
import re
found = []
match = re.compile('^(\S+).*?''(pathway)')
with open("997_from uniprot.gff", "r") as f:
    searchlines = f.readlines()
j=len(searchlines)-1 #with open('997_from uniprot.gff', "r") as raw:
for i, line in enumerate(searchlines):  #  for rec in raw:
        found.append(line)#line)#
#print(pd.DataFrame(found).shape)
#pd.DataFrame(found).head()
pd.DataFrame(found)

In [ ]:
'''with open("997_from uniprot.gff", "r") as f:
    searchlines = f.readlines()
    for i, line in enumerate(searchlines):
        print(line)'''
found = []
match = re.compile('^(\S+).*?(\w*)'" "'(signal)')
with open("997_from uniprot.gff", "r") as f:
    searchlines = f.readlines()
j=len(searchlines)-1 #with open('997_from uniprot.gff', "r") as raw:
for i, line in enumerate(searchlines):  #  for rec in raw:
    found.extend(match.findall(line))
print(pd.DataFrame(found).shape)
pd.DataFrame(found).head() 

In [ ]:
columns_names = ['Entry','one','two','three','f','fi','si','se','word_search','none']
uni_signal = pd.read_csv('signaling_pathway_uniprot.csv',sep="\t",header= None)
uni_signal.rename(columns={int(0):'Entry',int(8):'word_search'}, inplace=True)
uni_signal.drop(uni_signal.columns[[1,2,3,4,5,6,7,9]], axis=1, inplace=True)
print(uni_signal.shape)
uni_signal.head()

### <font color='green'> Extracting signaling pathway from Reactome dataset </font>


In [15]:
reactome = pd.read_csv('UniProt2Reactome.txt',sep="\t",header= None)
print(reactome.shape)
reactome.rename(columns={int(0):'Entry',int(3):'Reactome'}, inplace=True)
#pd.merge(reactome,s3 ,how='inner', on=['Entry'])
reactome.head(3)

(297048, 6)


,Entry,1,2,Reactome,4,5
0,A0A023GRW3,R-DME-72163,https://reactome.org/PathwayBrowser/#/R-DME-72163,mRNA Splicing - Major Pathway,IEA,Drosophila melanogaster
1,A0A023IKK2,R-RNO-202424,https://reactome.org/PathwayBrowser/#/R-RNO-20...,Downstream TCR signaling,IEA,Rattus norvegicus
2,A0A023IKK2,R-RNO-202427,https://reactome.org/PathwayBrowser/#/R-RNO-20...,Phosphorylation of CD3 and TCR zeta chains,IEA,Rattus norvegicus


In [137]:
match_reactome = reactome.loc[reactome['Entry'].isin(s3['Entry'])]
pos_match = match_reactome.groupby(['Entry'], as_index=False)['Reactome'].agg(",".join)

In [17]:
pos_match.head()

,Entry,Reactome
0,O00206,"ER-Phagosome pathway,Caspase activation via De..."
1,O00220,Caspase activation via Death Receptors in the ...
2,O00241,"DAP12 interactions,Signal regulatory protein f..."
3,O00478,Butyrophilin (BTN) family interactions
4,O00481,Butyrophilin (BTN) family interactions


In [ ]:
#uni_signal["word_search"].str.split("signaling pathway", n = 1, expand = True)     #data["Team"].str.rsplit("t", n = 1, expand = False)

### merging files for complete dataset

In [93]:
Reactome_signal_pathway=pd.merge(s3,pos_match ,how='inner', on=['Entry'])
print(Reactome_signal_pathway.shape)
Reactome_signal_pathway.to_csv(r'Dataset_with_reactome.csv', index = None, header=True)#(
Reactome_signal_pathway.head(2)

(438, 11)


,positionID,Entry,sequences,Protein names,Gene names,Seq_length,Protein Expression : High,Protein Expression : Low,Protein Expression : Med,Not detected,Reactome
0,P12821[30-1256],P12821,LDPGLQPGNFSADEAGAQLFAQSYNSSAEQVLFQSVAASWAHDTNI...,Angiotensin-converting enzyme (ACE) (EC 3.2.1....,ACE DCP DCP1,1227,"glandular cells, glandular cells, glandular ce...",endothelial cells,"glandular cells, pneumocytes, cells in seminif...","fibroblasts, glandular cells, peripheral nerve...",Metabolism of Angiotensinogen to Angiotensins
1,Q9BYF1[18-740],Q9BYF1,QSTIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMN...,Angiotensin-converting enzyme 2 (EC 3.4.17.23)...,ACE2 UNQ868/PRO1885,723,"glandular cells, Leydig cells, cells in semini...","glandular cells, glandular cells, glandular ce...",NaN,"adipocytes, smooth muscle cells, epidermal cel...",Metabolism of Angiotensinogen to Angiotensins


In [88]:
X_talk_db_sorted = X_talkprot2[['Entry', 'yes']]
X_talk_db_sorted.rename(columns={'yes':'X_talk pathway'},inplace=True)
X_talk_signal_pathway=pd.merge(s3, X_talk_db_sorted ,how='inner', on=['Entry'])
X_talk_signal_pathway.to_csv(r'Dataset_with_X_talk.csv', index = None, header=True)#(r'/Users/saheeba/Downloads/notlisted_extracellular.csv',)
print(X_talk_signal_pathway.shape)
X_talk_signal_pathway.head(2)

(21, 11)


,positionID,Entry,sequences,Protein names,Gene names,Seq_length,Protein Expression : High,Protein Expression : Low,Protein Expression : Med,Not detected,X_talk pathway
0,O14672[20-672],O14672,QYGNPLNKYIRHYEGLSYNVDSLHQKHQRAKRAVSHEDQFLRLDFH...,Disintegrin and metalloproteinase domain-conta...,ADAM10 KUZ MADM,653,NaN,"glandular cells, decidual cells, squamous epit...","fibroblasts, glandular cells, glandular cells,...","myocytes, glandular cells, trophoblastic cells...","Notch signaling pathway, Notch signaling pathway"
1,Q13145[21-152],Q13145,VLLTKGEIRCYCDAAHCVATGYMCKSELSACFSRLLDPQNSNSPLT...,BMP and activin membrane-bound inhibitor homol...,BAMBI NMA,132,NaN,"glandular cells, glandular cells, decidual cel...","glandular cells, respiratory epithelial cells,...","glandular cells, trophoblastic cells, ovarian ...",TGF-beta signaling pathway


In [73]:
#['bar'].fillna(df['foo'], inplace=True)
#del df['foo']
SignaLink_source=pd.DataFrame(source_match.rename(columns={'Source':'Entry','source_pathways':'SignaLink Pathway'}, inplace=True))
SignaLink_target = pd.DataFrame(target_match.rename(columns={'Target':'Entry','target_pathways':'SignaLink Pathway'}, inplace=True))
#print(SignaLink_source.iloc[:,1].tolist())#
SignaLink_source.append(SignaLink_target)
SignaLink_source.rename(columns={ 0:'SignaLink'},inplace=True)
#SignaLink_merge_pathway=pd.merge(s3, SignaLink_source ,how='inner', on=['Entry'])
#print(SignaLink_merge_pathway.shape)
#SignaLink_merge_pathway.head()

In [94]:
SignaLink_source.to_csv(r'SignaLink_source.csv')
SignaLink_match = pd.read_csv("SignaLink_source.csv")
SignaLink_match.rename(columns={'Source':'Entry'},inplace=True)
SignaLink_merge_pathway=pd.merge(s3, SignaLink_match ,how='inner', on=['Entry'])
SignaLink_merge_pathway = SignaLink_merge_pathway.drop(columns=['source_pathways'])
SignaLink_merge_pathway.to_csv(r'Dataset_with_SignaLink.csv', index = None, header=True)
print(SignaLink_merge_pathway.shape)

(60, 11)


In [129]:
#pd.merge(SignaLink_merge_pathway,X_talk_db_sorted ,how='outer', on=['Entry'])
together = pd.concat([SignaLink_merge_pathway, X_talk_signal_pathway,Reactome_signal_pathway], sort=True)#.shape
together.to_csv(r"together_all_pathway.csv")
print(together.shape)
together.head(1)


(519, 13)


,Entry,Gene names,Not detected,Protein Expression : High,Protein Expression : Low,Protein Expression : Med,Protein names,Reactome,Seq_length,SignaLink,X_talk pathway,positionID,sequences
0,Q04771,ACVR1 ACVRLK2,"cells in endometrial stroma, glandular cells, ...","glandular cells, smooth muscle cells","squamous epithelial cells, glial cells, pneumo...","myocytes, glandular cells, glandular cells, gl...",Activin receptor type-1 (EC 2.7.11.30) (Activi...,NaN,103,"TGF(core),TGF(core)",NaN,Q04771[21-123],MEDEKPKVNPKLYMCVCEGLSCGNEDHCEGQQCFSSLSINDGFHVY...


In [130]:
together['Reactome'].fillna(together['SignaLink'], inplace=True)
del together['SignaLink']
print(together.shape)
together.head(5)

(519, 12)


,Entry,Gene names,Not detected,Protein Expression : High,Protein Expression : Low,Protein Expression : Med,Protein names,Reactome,Seq_length,X_talk pathway,positionID,sequences
0,Q04771,ACVR1 ACVRLK2,"cells in endometrial stroma, glandular cells, ...","glandular cells, smooth muscle cells","squamous epithelial cells, glial cells, pneumo...","myocytes, glandular cells, glandular cells, gl...",Activin receptor type-1 (EC 2.7.11.30) (Activi...,"TGF(core),TGF(core)",103,NaN,Q04771[21-123],MEDEKPKVNPKLYMCVCEGLSCGNEDHCEGQQCFSSLSINDGFHVY...
1,P36896,ACVR1B ACVRLK4 ALK4,"melanocytes, Langerhans, keratinocytes, fibrob...",NaN,"glandular cells, decidual cells, squamous epit...","exocrine glandular cells, glandular cells, gla...",Activin receptor type-1B (EC 2.7.11.30) (Activ...,"TGF(core),TGF(core)",103,NaN,P36896[24-126],SGPRGVQALLCACTSCLQANYTCETDGACMVSIFNLDGMEHHVRTC...
2,Q8NER5,ACVR1C ALK7,Langerhans,"glandular cells, cells in tubules, glandular c...","glial cells, hematopoietic cells, adipocytes, ...","myoepithelial cells, cells in molecular layer,...",Activin receptor type-1C (EC 2.7.11.30) (Activ...,TGF(core),92,NaN,Q8NER5[22-113],LSPGLKCVCLLCDSSNFTCQTEGACWASVMLTNGKEQVIKSCVSLP...
3,Q13705,ACVR2B,"cells in endometrial stroma, cells in endometr...",chondrocytes,"endothelial cells, glandular cells, peripheral...","glandular cells, respiratory epithelial cells,...",Activin receptor type-2B (EC 2.7.11.30) (Activ...,"TGF(core),TGF(core),TGF(core),TGF(core),TGF(core)",119,NaN,Q13705[19-137],SGRGEAETRECIYYNANWELERTNQSGLERCEGEQDKRLHCYASWR...
4,P37023,ACVRL1 ACVRLK1 ALK1,"squamous epithelial cells, peripheral nerve/ga...","glandular cells, neuronal cells, cells in tubu...","glandular cells, glial cells, glandular cells,...","macrophages, cells in molecular layer, Purkinj...",Serine/threonine-protein kinase receptor R3 (S...,"TGF(core),TGF(core),TGF(core)",97,NaN,P37023[22-118],DPVKPSRGPLVTCTCESPHCKGPTCRGAWCTVVLVREEGRHPQEHR...


In [133]:
together['Reactome'].fillna(together['X_talk pathway'], inplace=True)
del together['X_talk pathway']
together.head(1)
print(together.shape)
together.head(5)

(519, 11)


,Entry,Gene names,Not detected,Protein Expression : High,Protein Expression : Low,Protein Expression : Med,Protein names,Reactome,Seq_length,positionID,sequences
0,Q04771,ACVR1 ACVRLK2,"cells in endometrial stroma, glandular cells, ...","glandular cells, smooth muscle cells","squamous epithelial cells, glial cells, pneumo...","myocytes, glandular cells, glandular cells, gl...",Activin receptor type-1 (EC 2.7.11.30) (Activi...,"TGF(core),TGF(core)",103,Q04771[21-123],MEDEKPKVNPKLYMCVCEGLSCGNEDHCEGQQCFSSLSINDGFHVY...
1,P36896,ACVR1B ACVRLK4 ALK4,"melanocytes, Langerhans, keratinocytes, fibrob...",NaN,"glandular cells, decidual cells, squamous epit...","exocrine glandular cells, glandular cells, gla...",Activin receptor type-1B (EC 2.7.11.30) (Activ...,"TGF(core),TGF(core)",103,P36896[24-126],SGPRGVQALLCACTSCLQANYTCETDGACMVSIFNLDGMEHHVRTC...
2,Q8NER5,ACVR1C ALK7,Langerhans,"glandular cells, cells in tubules, glandular c...","glial cells, hematopoietic cells, adipocytes, ...","myoepithelial cells, cells in molecular layer,...",Activin receptor type-1C (EC 2.7.11.30) (Activ...,TGF(core),92,Q8NER5[22-113],LSPGLKCVCLLCDSSNFTCQTEGACWASVMLTNGKEQVIKSCVSLP...
3,Q13705,ACVR2B,"cells in endometrial stroma, cells in endometr...",chondrocytes,"endothelial cells, glandular cells, peripheral...","glandular cells, respiratory epithelial cells,...",Activin receptor type-2B (EC 2.7.11.30) (Activ...,"TGF(core),TGF(core),TGF(core),TGF(core),TGF(core)",119,Q13705[19-137],SGRGEAETRECIYYNANWELERTNQSGLERCEGEQDKRLHCYASWR...
4,P37023,ACVRL1 ACVRLK1 ALK1,"squamous epithelial cells, peripheral nerve/ga...","glandular cells, neuronal cells, cells in tubu...","glandular cells, glial cells, glandular cells,...","macrophages, cells in molecular layer, Purkinj...",Serine/threonine-protein kinase receptor R3 (S...,"TGF(core),TGF(core),TGF(core)",97,P37023[22-118],DPVKPSRGPLVTCTCESPHCKGPTCRGAWCTVVLVREEGRHPQEHR...


In [136]:
together.drop_duplicates(keep='first')
together = together[['Entry','positionID','Gene names','Protein names','sequences','Seq_length','Protein Expression : High','Protein Expression : Med','Protein Expression : Low','Reactome']]
together.to_csv(r"519together_pathway.csv")
together.head(1)

,Entry,positionID,Gene names,Protein names,sequences,Seq_length,Protein Expression : High,Protein Expression : Med,Protein Expression : Low,Reactome
0,Q04771,Q04771[21-123],ACVR1 ACVRLK2,Activin receptor type-1 (EC 2.7.11.30) (Activi...,MEDEKPKVNPKLYMCVCEGLSCGNEDHCEGQQCFSSLSINDGFHVY...,103,"glandular cells, smooth muscle cells","myocytes, glandular cells, glandular cells, gl...","squamous epithelial cells, glial cells, pneumo...","TGF(core),TGF(core)"


In [117]:
together.to_csv(r"together_pathway.csv")